In [ ]:
import pandas as pd
import os
import google.generativeai as genai

In [5]:
df = pd.read_csv("parsed_output.csv")
df = df.dropna()

In [ ]:
# Configure the API key
genai.configure(api_key="YOUR API KEY")

# Load a Gemini model (text-only)
model = genai.GenerativeModel("gemini-2.5-flash")

def prompt_gemini(item, n):
# Send a prompt
    prompt = f"""
    Provide me the top {n} links for affordable {item}.
    Return the output as a single bracketed Python-style list, where each element is formatted as:
    ["WebsiteName", "URL"]

    Rules:
    - Use only one list — not multiple lines.
    - Separate entries with commas, not newlines.
    - Enclose the entire result in square brackets.
    - WebsiteName should be concise (e.g., "Pololu", "Adafruit").
    - URL must be the main home page (include https://).
    - No explanations, no extra text, just the list.
    """

    response = model.generate_content(prompt)

    return response

In [59]:
import json, ast, pandas as pd

top_n = 5

for index, row in df.iterrows():
    response = prompt_gemini(row["name"], top_n)

    raw_text = response.text.strip()

    # Convert string → Python list
    try:
        data = json.loads(raw_text)
    except json.JSONDecodeError:
        data = ast.literal_eval(raw_text)

    # Build DataFrame from parsed list
    df_top_choices = pd.DataFrame(data, columns=["Company", "Link"])

    display(df_top_choices)
    break


,Company,Link
0,AliExpress,https://www.aliexpress.com
1,Pololu,https://www.pololu.com
2,Banggood,https://www.banggood.com
3,StepperOnline,https://www.omc-stepperonline.com
4,Amazon,https://www.amazon.com
